# バリデーションデータによるハイパーパラメータのチューニング

## 検証(validation)<a name="description"></a>

- 機械学習ではモデルのパラメータはデータから自動的に学習するものとユーザーが設定するもの(モデルの引数に与える値)があるが、後者を特にハイパーパラメータと呼ぶことがある
- ハイパーパラメータをチューニングすることで精度を向上できるが、テストデータでの精度向上を目指してチューニングすると、手動でテストデータの情報をモデルに与え、テストデータに対して過学習を起こす
- ハイパーパラメータのチューニングには、データセットを訓練データ・バリデーションデータ・テストデータに分割し、訓練データで学習→バリデーションデータでハイパーパラメータをチューニング→テストデータで汎化能力確認という手順を踏む

### 検証データの作成<a name="data"></a>

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

loader = load_digits()
X, y = loader.data, loader.target
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=.5, random_state=0)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

### ハイパーパラメータのチューニング<a name="tuning"></a>

In [ ]:
import numpy as np
from sklearn.svm import SVC

best = {'score': 0}

for gamma in np.logspace(-6, 1, 8):
    model = SVC(gamma=gamma, random_state=0).fit(X_train, y_train)
    score = model.score(X_val, y_val)
    if score > best['score']:
        best = {'model': model, 'gamma': gamma, 'score': score}
print('Best score: {score:.3f} - Test score: {test:.3f} - Best param: gamma={gamma}'.format(
    score=best['score'], test=best['model'].score(X_test, y_test), gamma=best['gamma']))

### 交差検証(cross-validation)<a name="cross-validation"></a>

訓練データをk個に分割し、1個をバリデーションデータ・残りを訓練データとして利用し性能評価、それを全ての組で行って、平均性能をそのパラメータの性能とする。

- 訓練データとバリデーションデータの分け方による誤差に強い
- 各パラメータについてk回学習を行うので、実行にかかる時間が長くなる

完全にランダムにデータを分割すると、各グループでクラスの偏りが発生する可能性があるので、元データのクラス比率を維持しながら分割する層化k分割交差検証(stratified k-fold cross-validation)を用いる。

In [ ]:
# 検証曲線を表示すると、過学習の様子がわかりやすい(2本のグラフが乖離している部分が過学習)
from sklearn.model_selection import validation_curve
import matplotlib.pyplot as plt

X_cv, X_test, y_cv,y_test = train_test_split(X, y, test_size=.2, random_state=0, stratify=y)

param_range = np.logspace(-6, -1, 6)

# validation_curveのcvに整数を設定すると自動的にstratified k-fold cross-validationが適用される
train_scores, test_scores = validation_curve(estimator=SVC(random_state=0), X=X_cv, y=y_cv,
                                             param_name='gamma', param_range=param_range, cv=10, n_jobs=-1)

train_score_mean = train_scores.mean(axis=1)
test_score_mean = test_scores.mean(axis=1)

print('Best score: {score:.3f} - Best param: gamma={gamma}'.format(
    score=test_score_mean.max(), gamma=param_range[np.argmax(test_score_mean)]))

plt.plot(param_range, train_score_mean, label='Training')
plt.plot(param_range, test_score_mean, label='Validation')

plt.xscale('log')
plt.legend(loc='upper left')
plt.ylim(.7, 1.01)
plt.xlabel('gamma')
plt.ylabel('Accuracy')

plt.show()

In [ ]:
# GridSearchCVで最適パラメータを自動探索
from sklearn.model_selection import GridSearchCV

k=10
parameters = [
    {'kernel': ['linear'],
    'C': np.logspace(-3, 1, 5)},
    {'kernel': ['rbf'],
    'gamma': np.logspace(-6, 1, 8),
    'C': np.logspace(-3, 1, 5)}
]

# GridSearchCVのcvに整数を設定すると自動的にstratified k-fold cross-validationが適用される
clf = GridSearchCV(estimator=SVC(random_state=0), param_grid=parameters, n_jobs=-1, cv=k)
clf.fit(X_cv, y_cv)

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(clf.cv_results_)

In [ ]:
# クロスバリデーションでハイパーパラメータを決定後、全てのデータを使ってパラメータを学習したものがbest_estimatorに入る
print('Best score: {score:.3f} - Test score: {test:.3f} - Best param: {params}'.format(
    score=clf.best_score_, test=clf.best_estimator_.score(X_test, y_test),
    params=', '.join(['{k}={v}'.format(k=k, v=v) for k, v in clf.best_params_.items()])))